In [10]:
import os
from langchain_ollama import OllamaLLM
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_core.messages import SystemMessage

# Define AI models
classifier_model = OllamaLLM(model="mistral:latest")  # Updated AI Assistant (Classifier)
nlp_model = OllamaLLM(model="phi:latest")  # Natural Language Model
code_model = OllamaLLM(model="codellama:latest")  # Code Model (Fixed space issue)
checker = OllamaLLM(model="mistral:latest")  # Final Checker

# Define state class
class State(MessagesState):
    query: str
    category: str = None
    response: str = None


print(f"🧠 Using AI Assistant (Classifier): {classifier_model.model}")

🧠 Using AI Assistant (Classifier): mistral:latest


In [8]:
# Cargar variables de entorno
import os
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig

# Cargar variables de entorno
load_dotenv()

# Definir una herramienta para multiplicar
@tool
def multiplicar(a: int, b: int) -> str:
    """Devuelve el resultado de multiplicar a por b en formato texto."""
    resultado = a * b
    return f"El resultado de {a} * {b} es {resultado}."

# Inicializar el modelo LLM
llm = OllamaLLM(model="mistral:latest", temperature=0)

# Crear la consulta con uso de herramientas
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableLambda

# Definir un modelo de entrada para la herramienta
class MultiplicarInput(BaseModel):
    a: int = Field(..., description="Primer número a multiplicar")
    b: int = Field(..., description="Segundo número a multiplicar")

# Crear una función para usar la herramienta
def llamar_multiplicacion(inputs):
    return multiplicar.invoke(inputs)

# Convertir en una tarea ejecutable
multiplicar_runnable = RunnableLambda(llamar_multiplicacion)

# Hacer la consulta
pregunta = {"a": 4, "b": 56}
respuesta = multiplicar_runnable.invoke(pregunta)

print(respuesta)


El resultado de 4 * 56 es 224.


In [9]:
# Cargar variables de entorno
import os
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from langchain_core.agents import AgentExecutor, create_openai_tools_agent

# Cargar variables de entorno
load_dotenv()

# Definir una herramienta para multiplicar
@tool
def multiplicar(a: int, b: int) -> str:
    """Devuelve el resultado de multiplicar a por b en formato texto."""
    resultado = a * b
    return f"El resultado de {a} * {b} es {resultado}."

# Inicializar el modelo LLM
llm = OllamaLLM(model="mistral:latest", temperature=0)

# Definir herramientas disponibles para el agente
tools = [multiplicar]

# Crear el agente con herramientas
agent = create_openai_tools_agent(llm, tools)

# Crear el ejecutor del agente
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Hacer la consulta con el modelo
pregunta = "¿Cuánto es 4 * 56?"
respuesta = agent_executor.invoke({"input": pregunta})

print(respuesta)


ImportError: cannot import name 'AgentExecutor' from 'langchain_core.agents' (/home/jorgeglezdiaz/Escritorio/practises/langgraph/.venv/lib/python3.12/site-packages/langchain_core/agents.py)